In [2]:
from bettmensch_ai.pipelines import OutputArtifact, InputArtifact

from model import GPT1Core
from gpt_1 import get_source_data_split, get_tokenized_data_split_and_tokenizer, pretrain_and_checkpoint

In [6]:
get_source_data = False

if get_source_data:
    get_source_data_split(
        data_out=OutputArtifact(name="source_data_train")
    )

In [7]:
preprocess_data = True
N_TOTAL = 74004228
train_split = 0.9
n_train_observations = int(train_split * N_TOTAL)
sequence_length_test = 128

if preprocess_data:
    get_tokenized_data_split_and_tokenizer(
        source_data_split=InputArtifact(name="source_data_train"),
        end_index=n_train_observations,
        sequence_length=sequence_length_test,
        tokenized_data_out=OutputArtifact(name="tokenized_data_train"),
        tokenizer_out=OutputArtifact(name="tokenizer_train")
    )

    get_tokenized_data_split_and_tokenizer(
        source_data_split=InputArtifact(name="source_data_train"),
        start_index=n_train_observations,
        end_index=-1,
        sequence_length=sequence_length_test,
        tokenized_data_out=OutputArtifact(name="tokenized_data_validation"),
        tokenizer_out=OutputArtifact(name="tokenizer_validation")
    )

Number of observations in data set: 74004228
Size of data set in memory (in GB): 5.21540641784668e-08
2025-02-13 02:15:33 | Step 1/1332077
2025-02-13 02:15:47 | Step 2001/1332077
2025-02-13 02:15:48 | Step 4001/1332077
2025-02-13 02:15:51 | Step 6001/1332077
2025-02-13 02:15:52 | Step 8001/1332077
2025-02-13 02:15:54 | Step 10001/1332077
2025-02-13 02:15:55 | Step 12001/1332077
2025-02-13 02:15:56 | Step 14001/1332077
2025-02-13 02:15:57 | Step 16001/1332077
2025-02-13 02:15:58 | Step 18001/1332077
2025-02-13 02:15:59 | Step 20001/1332077
2025-02-13 02:16:00 | Step 22001/1332077
2025-02-13 02:16:02 | Step 24001/1332077
2025-02-13 02:16:03 | Step 26001/1332077
2025-02-13 02:16:04 | Step 28001/1332077
2025-02-13 02:16:06 | Step 30001/1332077
2025-02-13 02:16:07 | Step 32001/1332077
2025-02-13 02:16:08 | Step 34001/1332077
2025-02-13 02:16:09 | Step 36001/1332077
2025-02-13 02:16:10 | Step 38001/1332077
2025-02-13 02:16:11 | Step 40001/1332077
2025-02-13 02:16:13 | Step 42001/1332077
2025

Token indices sequence length is longer than the specified maximum sequence length for this model (1119 > 512). Running this sequence through the model will result in indexing errors


2025-02-13 02:17:40 | Step 172001/1332077
2025-02-13 02:17:41 | Step 174001/1332077
2025-02-13 02:17:42 | Step 176001/1332077
2025-02-13 02:17:43 | Step 178001/1332077
2025-02-13 02:17:44 | Step 180001/1332077
2025-02-13 02:17:46 | Step 182001/1332077
2025-02-13 02:17:47 | Step 184001/1332077
2025-02-13 02:17:48 | Step 186001/1332077
2025-02-13 02:17:49 | Step 188001/1332077
2025-02-13 02:17:50 | Step 190001/1332077
2025-02-13 02:17:51 | Step 192001/1332077
2025-02-13 02:17:52 | Step 194001/1332077
2025-02-13 02:17:53 | Step 196001/1332077
2025-02-13 02:17:54 | Step 198001/1332077
2025-02-13 02:17:55 | Step 200001/1332077
2025-02-13 02:18:42 | Step 202001/1332077
2025-02-13 02:18:43 | Step 204001/1332077
2025-02-13 02:18:45 | Step 206001/1332077
2025-02-13 02:18:46 | Step 208001/1332077
2025-02-13 02:18:47 | Step 210001/1332077
2025-02-13 02:18:48 | Step 212001/1332077
2025-02-13 02:18:49 | Step 214001/1332077
2025-02-13 02:18:50 | Step 216001/1332077
2025-02-13 02:18:51 | Step 218001/

Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors


2025-02-13 02:55:20 | Step 6001/148009
2025-02-13 02:55:21 | Step 8001/148009
2025-02-13 02:55:22 | Step 10001/148009
2025-02-13 02:55:23 | Step 12001/148009
2025-02-13 02:55:24 | Step 14001/148009
2025-02-13 02:55:25 | Step 16001/148009
2025-02-13 02:55:27 | Step 18001/148009
2025-02-13 02:55:28 | Step 20001/148009
2025-02-13 02:55:29 | Step 22001/148009
2025-02-13 02:55:30 | Step 24001/148009
2025-02-13 02:55:31 | Step 26001/148009
2025-02-13 02:55:32 | Step 28001/148009
2025-02-13 02:55:33 | Step 30001/148009
2025-02-13 02:55:34 | Step 32001/148009
2025-02-13 02:55:35 | Step 34001/148009
2025-02-13 02:55:36 | Step 36001/148009
2025-02-13 02:55:38 | Step 38001/148009
2025-02-13 02:55:39 | Step 40001/148009
2025-02-13 02:55:40 | Step 42001/148009
2025-02-13 02:55:41 | Step 44001/148009
2025-02-13 02:55:42 | Step 46001/148009
2025-02-13 02:55:43 | Step 48001/148009
2025-02-13 02:55:44 | Step 50001/148009
2025-02-13 02:55:45 | Step 52001/148009
2025-02-13 02:55:46 | Step 54001/148009
20

In [ ]:
pretrain = True

dim_embed_test = 768
n_decoder_layers_test = 12
n_heads_test = 12
dropout_test = 0.1
n_epochs_test = 1
batch_size_test = 32
display_step_test = 1

if pretrain:
    pretrain_and_checkpoint(
        tokenized_train=InputArtifact(name="tokenized_data_train"),
        tokenized_validation=InputArtifact(name="tokenized_data_validation"),
        tokenizer=InputArtifact(name="tokenizer_train"),
        sequence_length=sequence_length_test,
        dim_embed=dim_embed_test,
        n_decoder_layers=n_decoder_layers_test,
        n_heads=n_heads_test,
        dropout=dropout_test,
        n_epochs=n_epochs_test,
        batch_size=batch_size_test,
        display_step=display_step_test,
        verbose=False,
    )

EPOCH 1:
2025-02-13 03:09:04 | Batch 1/234394 (observations 0-32/7500608) loss: 155.1437530517578
2025-02-13 03:09:09 | Batch 2/234394 (observations 32-64/7500608) loss: 152.12014770507812
2025-02-13 03:09:13 | Batch 3/234394 (observations 64-96/7500608) loss: 147.095947265625
2025-02-13 03:09:17 | Batch 4/234394 (observations 96-128/7500608) loss: 144.5157928466797
2025-02-13 03:09:21 | Batch 5/234394 (observations 128-160/7500608) loss: 140.95401000976562
2025-02-13 03:09:25 | Batch 6/234394 (observations 160-192/7500608) loss: 137.2130584716797
2025-02-13 03:09:29 | Batch 7/234394 (observations 192-224/7500608) loss: 133.74241638183594
2025-02-13 03:09:33 | Batch 8/234394 (observations 224-256/7500608) loss: 131.276611328125
2025-02-13 03:09:37 | Batch 9/234394 (observations 256-288/7500608) loss: 125.53375244140625
2025-02-13 03:09:41 | Batch 10/234394 (observations 288-320/7500608) loss: 121.58838653564453
2025-02-13 03:09:45 | Batch 11/234394 (observations 320-352/7500608) loss: 

In [4]:
gpt_core = GPT1Core(40000)

In [5]:
gpt_core

GPT1Core(
  (embedding): Embedding(
    (embed): Embedding(40000, 768)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder): Decoder(
    (layers): ModuleList(
      (0-11): 12 x DecoderLayer(
        (skip_norm_mha): SkipNorm(
          (dropout): Dropout(p=0.1, inplace=False)
          (skipped_layer): MultiHeadedAttention(
            (W_queries): ModuleList(
              (0-11): 12 x Linear(in_features=768, out_features=64, bias=True)
            )
            (W_keys): ModuleList(
              (0-11): 12 x Linear(in_features=768, out_features=64, bias=True)
            )
            (W_values): ModuleList(
              (0-11): 12 x Linear(in_features=768, out_features=64, bias=True)
            )
            (W_out): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (norm_layer): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (skip_norm_ff): SkipNorm(
          (dropou

In [6]:
pytorch_total_params = sum(p.numel() for p in gpt_core.parameters() if p.requires_grad)
pytorch_total_params

116167680